# The Return of Superman Data


1. Load data from [Wikipedia](https://en.wikipedia.org/wiki/List_of_The_Return_of_Superman_episodes) and other sources (TVDB)
2. Clean Data

In [1]:
import requests
websiteURL = requests.get('https://en.wikipedia.org/wiki/List_of_The_Return_of_Superman_episodes').text

In [2]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(websiteURL, 'lxml')
print(soup.prettify()) # check content of soup

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of The Return of Superman episodes - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_The_Return_of_Superman_episodes","wgTitle":"List of The Return of Superman episodes","wgCurRevisionId":923264424,"wgRevisionId":923264424,"wgArticleId":50285268,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CS1 Korean-language sources (ko)","Articles with short description","Articles needing additional references from April 2016","All articles needing additional references","Lists of variety television series episodes","Lists of South Korean television series episodes"],"wgBreakFrames":!1,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"w

Get all wikitable plainrowheaders and save it to `listOfTables`

In [3]:
listOfTables = soup.findAll('table', {'class':'wikitable plainrowheaders'})
listOfTables

[<table class="wikitable plainrowheaders" style="width:99%;">
 <tbody><tr>
 <th scope="col" style="background-color: #B3003F; color: #FFFFFF;" width="7%">No. in<br/>season
 </th>
 <th scope="col" style="background-color: #B3003F; color: #FFFFFF;">Title
 </th>
 <th scope="col" style="background-color: #B3003F; color: #FFFFFF;" width="20%">Original air date
 </th>
 <th scope="col" style="background-color: #B3003F; color: #FFFFFF;" width="7%">Rating<sup class="reference" id="cite_ref-abg_1-0"><a href="#cite_note-abg-1">[1]</a></sup>
 </th></tr>
 <tr class="vevent" style="text-align:center;background:#F2F2F2"><th id="ep1" rowspan="1" scope="row" style="text-align:center">1</th><td class="summary" style="text-align:left">"A Man Became A Dad"</td><td style="text-align:center">September 19, 2013<span style="display:none"> (<span class="bday dtstart published updated">2013-09-19</span>)</span></td><td style="text-align:center">8.1%</td></tr><tr class="expand-child"><td class="description" cols

In [4]:
len(listOfTables)

8

There are 8 tables corresponding to each year of airing of The Return of Superman as listed in [Wikipedia](https://en.wikipedia.org/wiki/List_of_The_Return_of_Superman_episodes).  

---

In [5]:
import re 
Episode = []
Summary = []
Date = []
Rating = []
Description = []
GuestStar = []

for table in listOfTables:
    Episode = Episode + [item.get_text() for item in table.findAll('th', {'scope': 'row'})] # get episode number
    
    wholeRow = table.findAll('tr', {'class':'vevent'}) # get whole row of episode no., title, orig. air date, and rating
    emptyDescriptionsIndex = []
    
    indexCtr = 0
    for row in wholeRow:
        #print(row.get_text())
        Summary.append(row.find('td', {'class':'summary'}).get_text())
        temp = row.findAll('td', {'style':'text-align:center'})
        Date.append(temp[0].find('span', {'class':'bday dtstart published updated'}).get_text())
        Rating.append(temp[1].get_text())
        
        if row.next_sibling == "\n":
            emptyDescriptionsIndex.append(indexCtr)
            # episodes 233-235
            # emptyDescriptionsIndex contains the indices of the episodes with empty Description
        indexCtr = indexCtr + 1

    getDescription = table.findAll('tr', {'class': 'expand-child'})


    descriptionCtr = 0
    for item in getDescription:
        if descriptionCtr in emptyDescriptionsIndex:
            Description.append(None)
            GuestStar.append(None)
        
        itemFindP = item.find('p')
        if itemFindP == None:
            if item.find('td') != None:
                theDescription = item.find('td').get_text().strip()
                findGuestInDescription = re.search(r'(^$|.*)Guest star[s]*\(*s\)* *\(*.*\)*:[ .]*', theDescription)
                
                
                if findGuestInDescription is not None:
                    indexToCut = findGuestInDescription.start()
                    #print(indexToCut, "********")
                    Description.append(theDescription[:indexToCut])
                    temp_str = theDescription[indexToCut:] # contains guest star:

                    GuestStar.append(temp_str[temp_str.find(':')+1:])
                    #else:
                    #    Description.append(findGuestInDescription[0])
                    #print(re.search(r'Guest star[s]*\(*s\)* *\(*.*\)*:(.*)', theDescription))
                    #GuestStar.append(re.findall(r'Guest star[s]*\(*s\)* *\(*.*\)*:(.*)', theDescription))    
                else:
                    Description.append(theDescription)
                    GuestStar.append(None)
            else:
                Description.append(None)
            
        else:
            
            itemGetText = item.get_text().strip()
            findGuestInDescription = re.search(r'(^$|.*)Guest star[s]*\(*s\)* *\(*.*\)*:[ .]*', itemGetText)
            if findGuestInDescription is not None:
                indexToCut = findGuestInDescription.start()
                #print(indexToCut, "********")
                Description.append(itemGetText[:indexToCut])
                temp_str = itemGetText[indexToCut:] # contains guest star:
                
                GuestStar.append(temp_str[temp_str.find(':')+1:])
                #print(temp_str[(temp_str.find(':')+1):])
                #print("*_*")
            else:
                Description.append(itemGetText)
                truthSpecial = re.search(r'Special appearance\(*s\)*: (.*)', itemGetText) 
                if truthSpecial:
                    indexToCut = truthSpecial.start()
                    #print(indexToCut, "********")
                    temp_str = itemGetText[indexToCut:] # contains guest star:
                    print(itemGetText)
                    GuestStar.append(temp_str[temp_str.find(':')+1:])
                else:
                    GuestStar.append(None)
            
            #print(itemGetText)
            
            #print(itemGetText[:findGuestInDescription])
            #print("--")
            # sampleGuestStars = getGuestStarFunc(itemFindP)
            # Some guests are prefixed with "Guest stars: ", "Guest star(s): ", "Special appearance(s): ", etc.
            
            #truthGuest = re.findall(r'Guest star[s]*\(*s\)* *\(*.*\)*:(.*)', itemGetText)
            # truthSpecial = re.findall(r'Special appearance\(*s\)*: (.*)', itemGetText) 

            #if len(truthGuest) != 0 and truthGuest[0] != "":
            #    GuestStar.append(truthGuest)
            #elif len(truthGuest) == 0 and len(truthSpecial) != 0:
            #    GuestStar.append(truthSpecial)
            #else:
            #    GuestStar.append(None)
        descriptionCtr = descriptionCtr + 1
        print(descriptionCtr)
    

1
2
3
1
2
3
4
5
6
7
8
9
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
The naughty boys of the show Seoeon and Seojun! And the charming twin girls Seola and Sua! They meet once again! Also, Hwijae has stolen the heart of Daebak! and Donggook gets weary of the situation. Bumsoo and Daeul have a father-and-son date! They go to the car wash and dress up in suits! For Gyojin, the other two dads visit the drama reading room as a surprise! Everyone gets surprised at the visit of cute baby girls.

Special appearance(s): Kang Ye-won
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32


In [6]:
import pandas as pd
Episodes_df = pd.DataFrame()

Season = pd.Series([0] * len(Episode))
Season[3:] = 1
print(Season)
Episodes_df = pd.DataFrame()

#Episodes_df["season"] = [0] * 303
#Episodes_df.loc[0:2, "season"] = 0
#Episodes_df.loc[3:, "season"] = 1
#Episodes_df

0      0
1      0
2      0
3      1
4      1
      ..
299    1
300    1
301    1
302    1
303    1
Length: 304, dtype: int64


In [7]:
Episodes_df["season"] = Season
Episodes_df["episodeNumber"] = Episode
Episodes_df["episodeName"] = Summary
Episodes_df["dateAired"] = Date
Episodes_df["description"] = Description
Episodes_df["guestStar"] = GuestStar
Episodes_df["rating"] = Rating

len(Episodes_df)
Episodes_df

,season,episodeNumber,episodeName,dateAired,description,guestStar,rating
0,0,1,"""A Man Became A Dad""",2013-09-19,The four celebrity dads are introduced along w...,None,8.1%
1,0,2,"""The Dad's Are In Crisis""",2013-09-20,Lee Hwijae takes his twin boys out to a friend...,None,8.6%
2,0,3,"""In The Name of Father...""",2013-09-21,Lee Hwijae's father pays a visit to his home t...,None,8.5%
3,1,1,"""Things You Only Realize Once You Raise Them""",2013-11-03,Choo Sunghoon and Sarang are going to Osaka to...,"Jung Eun-pyo, Mithra Jin & DJ Tukutz from Epi...",7.2%
4,1,2,"""They’re Just Like Me""",2013-11-10,"Sunghoon and Sarang have so much fun in Osaka,...","Park Seo-joon, Yoon Bora, Lee Dong-wook",7.9%
...,...,...,...,...,...,...,...
299,1,297,"""Walking Through Memories""",2019-09-29,,,14.5%
300,1,298,"""When I'm with You, I Can Do It""",2019-10-06,,Jacob Batalon[51][52],14.1%
301,1,299,"""Let's Walk Our Autumn""",2019-10-13,,,14.3%
302,1,300,"""I Love You 300""",2019-10-20,,Lee Young-ja[53][54],13.9%


In [8]:
Episodes_df.loc[128, 'description']

"A handsome actor is here to take care of the five children of Donggook. The sisters are shy but Daebak seems to be nervous. A big rabbit is running around Seoeon and Seojun's house. He was expecting a big welcome but the twins ignore him. Two idol uncles appear in front of SoDa Siblings as their mom and dad. The two uncles can't help but surrender to the untiring siblings. Singer uncles have visited to see cute Rohui. They take care of Rohui just as they learned from Taeyoung. And they go out to take the subway for an outing. \n\n"

## Cleaning Data

Clean data per column

Procedure:
1. Check dtype of column and change dtype if it doesn't match with corresponding value type in the column
2. Find irregularities 

### episodeNumber column

In [9]:
print((Episodes_df['episodeNumber'].dtype))

object


In [10]:
Episodes_df[290:300]

,season,episodeNumber,episodeName,dateAired,description,guestStar,rating
290,1,288,"""Fly to Your World""",2019-07-28,,Lee Yong; Kim Na-young and Shin-woo Special a...,15.1%
291,1,289,"""Memories of Summer""",2019-08-04,,None.,14.3%
292,1,290,"""The Story Dad Didn't Know About""",2019-08-11,,Lee Yong Special appearance(s): Choi Chul-so...,14.7%
293,1,291,"""The Hottest Day of My Life""",2019-08-18,,Do Kyung-hwan Special appearance(s): Shin Ye...,15.3%
294,1,292,"""To You Who Shines Today""",2019-08-25,,"Lee Yong, Seong-gong (Ryu Seung-min's son), C...",15.1%
295,1,293,"""Can You Translate How I Feel?""",2019-09-01,,None.,15.4%
296,1,294,"""It's Written as Failure, but Reads as Experie...",2019-09-08,,"Lee Yong, Seong-gong (Ryu Seung-min's son), C...",16.0%
297,1,295 (Chuseok Special),"""Dear Moon, Our Love""",2019-09-15,,None.,14.5%
298,1,296,"""Today Is Another Lovely Day""",2019-09-22,,"Jeong Ju-ri [ko], Do-yun, Do-won and Do-ha",17.2%
299,1,297,"""Walking Through Memories""",2019-09-29,,,14.5%


Notice that the 295th episode on column 'episode Number' has Chuseok Special written write beside it. We will remove this and trasnfer this to another column named 'note' for later use and change 'episodeNumber's dtype to numeric.

In [11]:
Episodes_df.at[297, 'note'] = "Chuseok Special"
Episodes_df.loc[290:300]
Episodes_df.at[297, 'episodeNumber'] = 295
Episodes_df['episodeNumber'] = pd.to_numeric(Episodes_df['episodeNumber'])

### episodeName column

In [12]:
Episodes_df.head()

,season,episodeNumber,episodeName,dateAired,description,guestStar,rating,note
0,0,1,"""A Man Became A Dad""",2013-09-19,The four celebrity dads are introduced along w...,None,8.1%,NaN
1,0,2,"""The Dad's Are In Crisis""",2013-09-20,Lee Hwijae takes his twin boys out to a friend...,None,8.6%,NaN
2,0,3,"""In The Name of Father...""",2013-09-21,Lee Hwijae's father pays a visit to his home t...,None,8.5%,NaN
3,1,1,"""Things You Only Realize Once You Raise Them""",2013-11-03,Choo Sunghoon and Sarang are going to Osaka to...,"Jung Eun-pyo, Mithra Jin & DJ Tukutz from Epi...",7.2%,NaN
4,1,2,"""They’re Just Like Me""",2013-11-10,"Sunghoon and Sarang have so much fun in Osaka,...","Park Seo-joon, Yoon Bora, Lee Dong-wook",7.9%,NaN


To simplify the episodeName, let's remove the quotes.

In [13]:
Episodes_df['episodeName'] = Episodes_df['episodeName'].replace('\"','', regex=True)
Episodes_df.head()

,season,episodeNumber,episodeName,dateAired,description,guestStar,rating,note
0,0,1,A Man Became A Dad,2013-09-19,The four celebrity dads are introduced along w...,None,8.1%,NaN
1,0,2,The Dad's Are In Crisis,2013-09-20,Lee Hwijae takes his twin boys out to a friend...,None,8.6%,NaN
2,0,3,In The Name of Father...,2013-09-21,Lee Hwijae's father pays a visit to his home t...,None,8.5%,NaN
3,1,1,Things You Only Realize Once You Raise Them,2013-11-03,Choo Sunghoon and Sarang are going to Osaka to...,"Jung Eun-pyo, Mithra Jin & DJ Tukutz from Epi...",7.2%,NaN
4,1,2,They’re Just Like Me,2013-11-10,"Sunghoon and Sarang have so much fun in Osaka,...","Park Seo-joon, Yoon Bora, Lee Dong-wook",7.9%,NaN


In [14]:
Episodes_df['episodeName'].astype(str)

0                               A Man Became A Dad
1                          The Dad's Are In Crisis
2                         In The Name of Father...
3      Things You Only Realize Once You Raise Them
4                             They’re Just Like Me
                          ...                     
299                       Walking Through Memories
300                 When I'm with You, I Can Do It
301                          Let's Walk Our Autumn
302                                 I Love You 300
303                                            TBA
Name: episodeName, Length: 304, dtype: object

### dateAired column

In [15]:
print(Episodes_df['dateAired'].dtype)

object


In [16]:
Episodes_df.loc[170:180, 'dateAired']

170     2017-02-05
171     2017-02-12
172     2017-02-19
173    2017-02-026
174     2017-03-05
175     2017-03-12
176     2017-03-19
177     2017-03-26
178     2017-04-02
179     2017-04-09
180     2017-04-16
Name: dateAired, dtype: object

On the 173rd index, notice the date listed is not valid '2017-02-026'. 

If we immediately convert the type of 'dateAired' from object to datetime through the line of code below, it will throw an error because '2017-02-026' is of an unknown string format to `to_datetime`
```
Episodes_df['dateAired'] = pd.to_datetime(Episodes_df['dateAired'])
```
Let's remove the extra zero manually and change the type of 'dateAired' column to datetime.

In [17]:
Episodes_df.loc[173, 'dateAired'] = '2017-02-26'
Episodes_df['dateAired'] = pd.to_datetime(Episodes_df['dateAired'])
Episodes_df.loc[170:180, 'dateAired']

170   2017-02-05
171   2017-02-12
172   2017-02-19
173   2017-02-26
174   2017-03-05
175   2017-03-12
176   2017-03-19
177   2017-03-26
178   2017-04-02
179   2017-04-09
180   2017-04-16
Name: dateAired, dtype: datetime64[ns]

### description column

In [18]:
pd.set_option('display.max_colwidth', -1) # to display all the content within the dataframe set display.max_colwidth to -1
Episodes_df.loc[195:200,'description']

195    Taeyoung and Rohui prepare an event for the couple’s 6th anniversary and take a summer trip to a river with Eugene. Gyojin’s family welcomes visitors while Gyojin gets to have pregnancy experience to understand Ehyun better. Jiyong and Seungjae’s zone-out stay takes an unexpected turn. And Seojun and Seoeon learn about sportsmanship by competing with each other at the sports class.
196    Hwijae takes Seoeon and Seojun to World Insect Fair as well as Folk Village to have a good time. Donggook’s children spend a day with star uncles[18] while Donggook is out for work. William’s grandmother comes to visit Sam’s family from Australia, and goes on a Seoul tour with them.[19] Seungjae learns how to be a big brother by playing with his little cousin.\n\n                  
197    Jiyong and Seungjae visit Jiyong’s elementary school together. Sam and William spend a special day with William’s grandmother who is visiting for William’s first birthday. Hwijae takes the twins to a special l

We can remove the extra spaces/new lines through stripping the description.

In [19]:
Episodes_df['description'] = Episodes_df['description'].apply(lambda x: x.strip() if x else None)

In [20]:
Episodes_df.loc[195:200,'description']

195    Taeyoung and Rohui prepare an event for the couple’s 6th anniversary and take a summer trip to a river with Eugene. Gyojin’s family welcomes visitors while Gyojin gets to have pregnancy experience to understand Ehyun better. Jiyong and Seungjae’s zone-out stay takes an unexpected turn. And Seojun and Seoeon learn about sportsmanship by competing with each other at the sports class.
196    Hwijae takes Seoeon and Seojun to World Insect Fair as well as Folk Village to have a good time. Donggook’s children spend a day with star uncles[18] while Donggook is out for work. William’s grandmother comes to visit Sam’s family from Australia, and goes on a Seoul tour with them.[19] Seungjae learns how to be a big brother by playing with his little cousin.                      
197    Jiyong and Seungjae visit Jiyong’s elementary school together. Sam and William spend a special day with William’s grandmother who is visiting for William’s first birthday. Hwijae takes the twins to a special l

We can also remove the citations enclosed by square brackets used by Wikipedia. (Sample of citation mentioned can be seen in the 196th description above ex. [18], [19])

Let's remove the citations in the 'description' column.

In [21]:
columns = ['description']

for column in columns:
    #Episodes_df[column] = Episodes_df[column].str.replace(r"\(.*\)", "")
    Episodes_df[column] = Episodes_df[column].replace(r"\[.*\]", "")

In [22]:
Episodes_df.loc[195:200,'description']

195    Taeyoung and Rohui prepare an event for the couple’s 6th anniversary and take a summer trip to a river with Eugene. Gyojin’s family welcomes visitors while Gyojin gets to have pregnancy experience to understand Ehyun better. Jiyong and Seungjae’s zone-out stay takes an unexpected turn. And Seojun and Seoeon learn about sportsmanship by competing with each other at the sports class.
196    Hwijae takes Seoeon and Seojun to World Insect Fair as well as Folk Village to have a good time. Donggook’s children spend a day with star uncles[18] while Donggook is out for work. William’s grandmother comes to visit Sam’s family from Australia, and goes on a Seoul tour with them.[19] Seungjae learns how to be a big brother by playing with his little cousin.                      
197    Jiyong and Seungjae visit Jiyong’s elementary school together. Sam and William spend a special day with William’s grandmother who is visiting for William’s first birthday. Hwijae takes the twins to a special l

In [23]:
pd.set_option('display.max_colwidth', 50) # return column width of pandas to default = 50
Episodes_df.loc[10:13,'description']

10    The four dads continue with their taping of Ha...
11    Sunghoon and Sarang visit Mt. Fuji, but Sarang...
12    Sunghoon and Sarang decide to go to Okinawa to...
13    The Choovely family begin their trip in Okinaw...
Name: description, dtype: object

### guestStar column

In [24]:
Episodes_df['guestStar']

0                                                   None
1                                                   None
2                                                   None
3       Jung Eun-pyo, Mithra Jin & DJ Tukutz from Epi...
4                Park Seo-joon, Yoon Bora, Lee Dong-wook
                             ...                        
299                                                     
300                                Jacob Batalon[51][52]
301                                                     
302                                 Lee Young-ja[53][54]
303                                                     
Name: guestStar, Length: 304, dtype: object

In [25]:
for item in Episodes_df['guestStar']:
    if item:
        print(item)

 Jung Eun-pyo, Mithra Jin & DJ Tukutz from Epik High, Jeong Hyeong-don
 Park Seo-joon, Yoon Bora, Lee Dong-wook
 DJ Tukutz with his son Kim Yunu, Lee Young-Hyun, Jeong Hyeong-don via phone call, Jeong Jun-ha via phone call
 Lee Hyun-do, Choi Hong-man
 Yoo Jae-suk, Park Myung-su
 Yoo Jae-suk, Park Myung-su, Shin Bong-sun, Park Mi-sun, Hwang Kwanghee, Jung Eun-pyo and his family
 Jeong Jun-ha via phone call
 Hwang Jae-kyun (Lotte Giants infielder)
 Cha Tae-hyun via phone call, Hwang Jung-min via phone call
 Director Jang Hang-jun with his daughter Jang Yun-seo
 Rain
 Big Bang, Sandara Park from 2NE1, Lee Hi, Mithra Jin & DJ Tukutz from Epik High
 Kim Heechul & Kangin from Super Junior, Jeong Eun-pyo and his family, cast of Hwayi: A Monster Boy (Kim Yoon-seok, Cho Jin-woong)
 cast of Hwayi: A Monster Boy (Kim Yoon-seok, Cho Jin-woong, Kim Sung-kyun, Yeo Jin-goo), Woo Ji-won with his daughter Woo Na-yun, Kim Jung-tae with his son Kim Ji-hoo (Yakkung), Dong Hyun Kim
 Kim Jung-tae with his s

In [26]:
Episodes_df.loc[0:30, 'guestStar']


0                                                  None
1                                                  None
2                                                  None
3      Jung Eun-pyo, Mithra Jin & DJ Tukutz from Epi...
4               Park Seo-joon, Yoon Bora, Lee Dong-wook
5                                                  None
6                                                  None
7      DJ Tukutz with his son Kim Yunu, Lee Young-Hy...
8                            Lee Hyun-do, Choi Hong-man
9                            Yoo Jae-suk, Park Myung-su
10     Yoo Jae-suk, Park Myung-su, Shin Bong-sun, Pa...
11                          Jeong Jun-ha via phone call
12              Hwang Jae-kyun (Lotte Giants infielder)
13     Cha Tae-hyun via phone call, Hwang Jung-min v...
14     Director Jang Hang-jun with his daughter Jang...
15                                                 None
16                                                 None
17                                              

In [27]:
Episodes_df.dtypes

season                    int64
episodeNumber             int64
episodeName              object
dateAired        datetime64[ns]
description              object
guestStar                object
rating                   object
note                     object
dtype: object

In [28]:
pd.set_option('display.max_rows', 60, 'display.max_columns', 20)

In [29]:
Episodes_df['guestStar'].apply(lambda x: isinstance(x, list))

0      False
1      False
2      False
3      False
4      False
       ...  
299    False
300    False
301    False
302    False
303    False
Name: guestStar, Length: 304, dtype: bool

In [30]:
Episodes_df.at[137, 'guestStar']

' Kang Ye-won'

In [31]:
Episodes_df.loc[250:280]

,season,episodeNumber,episodeName,dateAired,description,guestStar,rating,note
250,1,248,You Can Do It/Changing Together Like Autumn Le...,2018-10-21,None,None,8.5%,NaN
251,1,249,From Mount Halla to Mount Paektu,2018-10-28,None,"Jo Sung-mo and Bong-yeon, Richard Windbichler",11.8%,NaN
252,1,250,From Mount Halla to Mount Paektu Part 2,2018-11-04,None,None,9.4%,NaN
253,1,251,From Mount Halla to Mount Paektu Part 3,2018-11-11,None,"Richard Windbichler, Sergio Escudero, Lee Myu...",10.9%,NaN
254,1,252,One Step at a Time Into the World,2018-11-18,None,None,9.4%,NaN
255,1,253,The Reason Why Life is Worth Living,2018-11-25,None,"Jo Sung-mo and Bong-yeon, Son Ho-young (g.o.d...",10.1%,NaN
256,1,254,I'll Go to You Like Winter's First Snow,2018-12-02,None,Key (Shinee)[46],10.8%,NaN
257,1,255,You're the Center of My Universe,2018-12-09,None,None,11.2%,NaN
258,1,256,A Winter Fairytale,2018-12-16,None,"Lee Hye-jung, Kim Ji-min",10.9%,NaN
259,1,257,The Miracle of December,2018-12-23,None,"Kai (EXO), Chanyeol (EXO),[47] Lee Yoon-ji an...",10.1%,NaN


Upon inspecting the guestStar column, we can remove all the unnecessary characters and words.

In [32]:
def replace_func(x):
    x = x.strip()

    replaceStr = (("and", ", "),
             ("&", ","),
             (";", ","),
             (u'\xa0', u' '),
             ("Special appearance(s):", ","),
             ("(brief appearance)", ""),
             ("None.", ""),
             ("None. ", ""))

    for r in replaceStr:
        x = x.replace(*r)
    
    replaceStr = ((r'(\[[A-Za-z0-9]+\])', ''),
              (r' +', ' '),
              (r'[ ]*,[ ]*', ','))
    
    for r in replaceStr:
        x = re.sub(*r, x)


    #x = x.strip()

    if len(x) != 0:
        return x
    else:
        return None
    
    
# df[['A','C']].apply(lambda x: my_func(x) if(np.all(pd.notnull(x[1]))) else x, axis = 1)
Episodes_df['guestStar'] = Episodes_df['guestStar'].apply(lambda x: replace_func(x) if x is not None else None)

In [33]:
#type(Episodes_df.at[298, 'Guest Star'])
#print(Episodes_df.at[0, 'Guest Star'])
Episodes_df['guestStar']

0                                                   None
1                                                   None
2                                                   None
3      Jung Eun-pyo,Mithra Jin,DJ Tukutz from Epik Hi...
4                  Park Seo-joon,Yoon Bora,Lee Dong-wook
                             ...                        
299                                                 None
300                                        Jacob Batalon
301                                                 None
302                                         Lee Young-ja
303                                                 None
Name: guestStar, Length: 304, dtype: object

In [34]:
myre = re.compile(r"[,]*([\w ]+)([\(]+[a-zA-Z.-]+,.*?[\)]+)")
cleanStarNames = []
for item in Episodes_df['guestStar']:
    if item is not None:
        cleanStarNames.append(myre.findall(item))
    else:
        cleanStarNames.append(None)

len(cleanStarNames)
clean2 = []
for item in cleanStarNames:
    if item:
        #item[0][0] = item[0][0].strip()
        strip2 = item[0][1].split(',')
        temp_list = []
        for item2 in strip2:
            temp_list.append(item2.replace("(", "").replace(")", "").strip() + " ("+item[0][0].strip() + ")")
        clean2.append(temp_list)
    else:
        clean2.append(None)

clean2
        #.split(',')
# split 1st level using parenthesis
# split 2nd level using comma
    

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 ['Taeyeon (Generation)',
  'Sunny (Generation)',
  'Tiffany (Generation)',
  'Sooyoung (Generation)',
  'Yoona (Generation)'],
 None,
 None,
 None,
 None,
 None,
 ['G-Dragon (Big Bang)', 'Taeyang (Big Bang)', 'Daesung (Big Bang)'],
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 Non

In [35]:
Episodes_df.loc[:,'guestStar'] = [item.split(',') if item != None else None for item in Episodes_df.loc[:,'guestStar']]

In [36]:
import numpy as np
#Episodes_df['Guest Star'] = 
Episodes_df['guestStar'] = np.where(clean2, clean2, Episodes_df['guestStar'])

In [37]:
#len(Episodes_df.at[292,'guestStar'])
Episodes_df.at[290,'guestStar']

['Lee Yong',
 'Kim Na-young',
 'Shin-woo',
 'Choi Chul-soon',
 'Jae-hee',
 'Lee Hyo-jung',
 'Bin',
 'Ryu Seung-min',
 'Seong-gong',
 'Dan-woo',
 "his sister (Moon So-ri 's children)",
 'Park Ga-won',
 "Yoo-joon (Kang Byeong-hyun 's son)",
 'Cho Dong-hwa',
 'Jae-beom',
 "Yoo Da-on (Yoo Won-sang's daughter)",
 'Chon Tae-poong (Tony Akins)',
 'Tae-yong']

In [38]:
Episodes_df.to_csv(r'samp.csv')

In [39]:
Episodes_df.loc[0:20]

,season,episodeNumber,episodeName,dateAired,description,guestStar,rating,note
0,0,1,A Man Became A Dad,2013-09-19,The four celebrity dads are introduced along w...,None,8.1%,NaN
1,0,2,The Dad's Are In Crisis,2013-09-20,Lee Hwijae takes his twin boys out to a friend...,None,8.6%,NaN
2,0,3,In The Name of Father...,2013-09-21,Lee Hwijae's father pays a visit to his home t...,None,8.5%,NaN
3,1,1,Things You Only Realize Once You Raise Them,2013-11-03,Choo Sunghoon and Sarang are going to Osaka to...,"[Jung Eun-pyo, Mithra Jin, DJ Tukutz from Epik...",7.2%,NaN
4,1,2,They’re Just Like Me,2013-11-10,"Sunghoon and Sarang have so much fun in Osaka,...","[Park Seo-joon, Yoon Bora, Lee Dong-wook]",7.9%,NaN
5,1,3,I Can't Live Without You,2013-11-10,Sunghoon and his dad want to teach Sarang judo...,None,8.4%,NaN
6,1,4,This is What Your Dad is Like,2013-11-24,Sarang's suffering from constipation. To show ...,None,6.8%,NaN
7,1,5,It’s Okay to Fail at First,2013-12-01,Haru learns to spend more time with Tablo's ba...,"[DJ Tukutz with his son Kim Yunu, Lee Young-Hy...",7.4%,NaN
8,1,6,Kids Raise Dads,2013-12-08,"Haru goes to a fishing park with Tablo, they c...","[Lee Hyun-do, Choi Hong-man]",7.5%,NaN
9,1,7,Sons & Daughters,2013-12-15,"Sarang's mother, Shiho Yano gives a ton of cho...","[Yoo Jae-suk, Park Myung-su]",8.8%,NaN


## Get data from tvdb database

In [40]:
import requests
import pandas as pd

token = ""
try:
    with open( "token.txt" ) as f :
        token = f.readlines()
except Exception:
     print('File token.txt not found')

#print(token)

url = ["https://api.thetvdb.com/series/276221/episodes",
    "https://api.thetvdb.com/series/276221/episodes?page=2",
    "https://api.thetvdb.com/series/276221/episodes?page=3",
    "https://api.thetvdb.com/series/276221/episodes?page=4"]

def get_with_token(url, query_params=None):
    headers = {
        "Accept": "application/json",
        "Authorization": f"Bearer {token[0]}",
    }
    return requests.get(url, headers=headers, params=query_params)

json_list = []
for item in url:
    response = get_with_token(item) 
    json_list.append(response)

#print(type(json_list[0].text))

In [41]:
json_list[0].text

'{"links":{"first":1,"last":4,"next":2,"prev":null},"data":[{"id":4737100,"airedSeason":1,"airedSeasonID":570102,"airedEpisodeNumber":1,"episodeName":"Things You Only Realize Once You Raise Them","firstAired":"2013-11-03","guestStars":["Jeong Eun-pyo, Mithra Jin \\u0026 DJ Tukutz from Epik High, Jeong Hyeong-don"],"director":"","directors":[],"writers":[],"overview":"The four celebrity dads are introduced along with their children. Lee Hyunwoo takes his sons to the market. Lee Hwijae has to struggle to take care of his 8 month old twin boys alone for the first time. Choo Sunghoon takes his 1 year old daughter out for an hair cut. Jang Hyunsung learns to cook and do housework for the first time in his life. \\r\\nGuest stars: Jeong Eun-pyo, Mithra Jin \\u0026 DJ Tukutz from Epik High, Jeong Hyeong-don","language":{"episodeName":"en","overview":"en"},"productionCode":"","showUrl":"","lastUpdated":1434761747,"dvdDiscid":"","dvdSeason":null,"dvdEpisodeNumber":null,"dvdChapter":null,"absolu

In [42]:
import json
from pandas.io.json import json_normalize

#jsonObj = json.loads(json_list[0].text)

frames = []
for item in json_list:
    jsonObj = json.loads(item.text)
    frames.append(json_normalize(jsonObj['data']))

df_episode = pd.concat(frames)
df_episode.overview

0     The four celebrity dads are introduced along w...
1     Sunghoon and Sarang have so much fun in Osaka,...
2     Sunghoon and his dad wants to teach Sarang jud...
3     Sarang's suffering from constipation. To show ...
4     Haru learns to spend more time with Tablo band...
                            ...                        
98    William and Bentley meet Jung Jury’s children....
99    Dong Gook and Si An go on a special father-son...
0     Gun Hoo and Na Eun go to a restaurant with the...
1     Hee Joon and Jam Jam planned a picnic, but due...
2     Hee Joon and Jam Jam go to a glamping site to ...
Name: overview, Length: 303, dtype: object

In [43]:
df_episode.columns

Index(['id', 'airedSeason', 'airedSeasonID', 'airedEpisodeNumber',
       'episodeName', 'firstAired', 'guestStars', 'director', 'directors',
       'writers', 'overview', 'productionCode', 'showUrl', 'lastUpdated',
       'dvdDiscid', 'dvdSeason', 'dvdEpisodeNumber', 'dvdChapter',
       'absoluteNumber', 'filename', 'seriesId', 'lastUpdatedBy',
       'airsAfterSeason', 'airsBeforeSeason', 'airsBeforeEpisode',
       'thumbAuthor', 'thumbAdded', 'thumbWidth', 'thumbHeight', 'imdbId',
       'siteRating', 'siteRatingCount', 'language.episodeName',
       'language.overview'],
      dtype='object')

In [44]:
df_new_episode = df_episode[['airedSeason', 'airedEpisodeNumber', 'episodeName', 'firstAired', 'guestStars', 'overview']]
df_new_episode

,airedSeason,airedEpisodeNumber,episodeName,firstAired,guestStars,overview
0,1,1,Things You Only Realize Once You Raise Them,2013-11-03,"[Jeong Eun-pyo, Mithra Jin & DJ Tukutz from Ep...",The four celebrity dads are introduced along w...
1,1,2,They're Just Like Me,2013-11-10,"[Park Seo-joon, Yoon Bora, Lee Dong-wook]","Sunghoon and Sarang have so much fun in Osaka,..."
2,1,3,I Can't Live Without You,2013-11-17,[],Sunghoon and his dad wants to teach Sarang jud...
3,1,4,This is What Your Dad is Like,2013-11-24,[],Sarang's suffering from constipation. To show ...
4,1,5,It's Okay to Fail at First,2013-12-01,[],Haru learns to spend more time with Tablo band...
...,...,...,...,...,...,...
98,1,296,Today Is Another Lovely Day,2019-09-22,[],William and Bentley meet Jung Jury’s children....
99,1,297,Walking Through Memories,2019-09-29,[],Dong Gook and Si An go on a special father-son...
0,1,298,"If I'm with You, I Can Do It",2019-10-06,[],Gun Hoo and Na Eun go to a restaurant with the...
1,1,299,Let's Walk Our Autumn,2019-10-13,[],"Hee Joon and Jam Jam planned a picnic, but due..."


In [45]:
print((Episodes_df['episodeNumber'].dtype))
print((df_new_episode['airedEpisodeNumber'].dtype))

int64
int64


In [46]:
Episodes_df.loc[0:2, 'season'] = 0
Episodes_df.loc[3:, 'season'] = 1

In [47]:
Episodes_df

,season,episodeNumber,episodeName,dateAired,description,guestStar,rating,note
0,0,1,A Man Became A Dad,2013-09-19,The four celebrity dads are introduced along w...,None,8.1%,NaN
1,0,2,The Dad's Are In Crisis,2013-09-20,Lee Hwijae takes his twin boys out to a friend...,None,8.6%,NaN
2,0,3,In The Name of Father...,2013-09-21,Lee Hwijae's father pays a visit to his home t...,None,8.5%,NaN
3,1,1,Things You Only Realize Once You Raise Them,2013-11-03,Choo Sunghoon and Sarang are going to Osaka to...,"[Jung Eun-pyo, Mithra Jin, DJ Tukutz from Epik...",7.2%,NaN
4,1,2,They’re Just Like Me,2013-11-10,"Sunghoon and Sarang have so much fun in Osaka,...","[Park Seo-joon, Yoon Bora, Lee Dong-wook]",7.9%,NaN
...,...,...,...,...,...,...,...,...
299,1,297,Walking Through Memories,2019-09-29,None,None,14.5%,NaN
300,1,298,"When I'm with You, I Can Do It",2019-10-06,None,[Jacob Batalon],14.1%,NaN
301,1,299,Let's Walk Our Autumn,2019-10-13,None,None,14.3%,NaN
302,1,300,I Love You 300,2019-10-20,None,[Lee Young-ja],13.9%,NaN


In [48]:
pd.merge(Episodes_df, df_new_episode, left_on=['episodeNumber', 'season'], right_on=['airedEpisodeNumber', 'airedSeason'], how='left').loc[280:300]

,season,episodeNumber,episodeName_x,dateAired,description,guestStar,rating,note,airedSeason,airedEpisodeNumber,episodeName_y,firstAired,guestStars,overview
280,1,278,Never-ending Cherry Blossom,2019-05-19,None,None,15.4%,NaN,1.0,278.0,Never-ending Cherry Blossom,2019-05-19,[],It’s Beom June family’s last episode today. He...
281,1,279,"Today, We Sing Again",2019-05-26,None,"[Hong Kyung-min, Ra-won, Ra-im]",15.2%,NaN,1.0,279.0,"Today, We Sing Again",2019-05-26,[],Joo Ho takes Na Eun and Gun Hoo out for campin...
282,1,280,Will You Remember Me?,2019-06-02,None,None,14.0%,NaN,1.0,280.0,Will You Remember Me?,2019-06-02,[],"It’s Seung Jae’s last day on the show, so Ji Y..."
283,1,281,To You Who Taught Me Happiness,2019-06-09,None,None,14.6%,NaN,1.0,281.0,To You who Taught Me Happiness,2019-06-09,[],"The very first idol couple, Moon Hee Joon and ..."
284,1,282,A Fairytale in Our Daily Life,2019-06-16,None,"[Park Jung-chul, Da-in, Hwang Kwang-hee]",15.0%,NaN,1.0,282.0,A Fairytale in Our Daily Life,2019-06-16,[],Hee Joon and Hee Yul go to the market and prep...
285,1,283,A Poem for the Little Things,2019-06-23,None,[Hwang Kwang-hee],15.2%,NaN,1.0,283.0,A Poem for the Little Things,2019-06-23,[],Re Won shows off her unusual taste as a kid. K...
286,1,284,I'm Happy When You Smile,2019-06-30,None,[Hwang Kwang-hee],15.9%,NaN,1.0,284.0,I'm Happy When You Smile,2019-06-30,[],Dong Gook's children have had enough of their ...
287,1,285,The Sound of the Approaching Summer,2019-07-07,None,"[Lee Seok-hoon (SG Wannabe), Joo-won, Hwang Kw...",15.8%,NaN,1.0,285.0,The Sound of Summer Approaching,2019-07-07,[],Hee Joon plants a tree with Hee Yul. They also...
288,1,286,We Must Stay Together to Live,2019-07-14,None,None,13.8%,NaN,1.0,286.0,We Must Stay Together to Live,2019-07-14,[],"Seol A, Soo A, and Si An go on a retreat with ..."
289,1,287,You're My Flower,2019-07-21,None,"[Moon Jeong-won, Lee Seo-eon, Lee Seo-jun, Lee...",13.6%,NaN,1.0,287.0,You're My Flower,2019-07-21,[],"Moon Jung Won, Seo Eon, and Seo Jun make a sur..."
